In [ ]:
import logging
from abc import ABC, abstractmethod
from datetime import datetime
from typing import List


logging.basicConfig(
    filename="AdvancedBankSystem.log",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    filemode="a",
    force=True
)

logging.info("Advanced Banking System Started")


class InsufficientBalanceError(Exception):
    """Custom exception for insufficient balance"""
    pass


class Account(ABC):
    """
    Abstract Base Class for all account types.

    Attributes:
        _account_number (int): Unique account number
        _holder_name (str): Account holder name
        __balance (float): Private account balance
        _transactions (List[str]): Stores transaction history
    """

    bank_name = "Panjab National Bank"
    minimum_balance = 1000

    def __init__(self, account_number: int, holder_name: str, balance: float) -> None:
        """
        Constructor for Account class

        Parameters:
            account_number (int): Unique account number
            holder_name (str): Name of account holder
            balance (float): Initial deposit amount

        Returns:
            None
        """
        self._account_number = account_number
        self._holder_name = holder_name
        self.__balance = balance
        self._transactions: List[str] = []

        logging.info(f"Account created for {holder_name}")

    def _get_balance(self) -> float:
        """
        Protected method to access private balance

        Returns:
            float: current balance
        """
        return self.__balance

    def _set_balance(self, amount: float) -> None:
        """
        Protected method to update balance safely

        Parameters:
            amount (float): New balance value

        Returns:
            None
        """
        self.__balance = amount

    def get_balance(self) -> float:
        """
        Returns the current account balance.

        Returns:
            float: current balance
        """
        return self.__balance

    def deposit(self, amount: float) -> None:
        """
        Deposit money into account.

        Parameters:
            amount (float): amount to deposit

        Returns:
            None

        Raises:
            ValueError: if amount is negative
        """
        if amount <= 0:
            raise ValueError("Deposit amount must be positive")

        self.__balance += amount
        self._transactions.append(
            f"{datetime.now()} - Deposited ₹{amount}"
        )
        logging.info(f"{amount} deposited in {self._account_number}")

    def withdraw(self, amount: float) -> None:
        """
        Withdraw money from account.

        Parameters:
            amount (float): amount to withdraw

        Returns:
            None

        Raises:
            InsufficientBalanceError: if balance is insufficient
        """
        if amount <= 0:
            raise ValueError("Withdraw amount must be positive")

        if self.__balance - amount < self.minimum_balance:
            raise InsufficientBalanceError(
                "Cannot withdraw. Minimum balance must be maintained."
            )

        self.__balance -= amount
        self._transactions.append(
            f"{datetime.now()} - Withdrawn ₹{amount}"
        )
        logging.info(f"{amount} withdrawn from {self._account_number}")

    def show_transactions(self) -> List[str]:
        """
        Returns transaction history.

        Returns:
            List[str]: list of transaction records
        """
        return self._transactions

    def account_details(self) -> str:
        """
        Returns formatted account details.

        Returns:
            str: account information
        """
        return (
            f"Bank Name: {self.bank_name}\n"
            f"Account Number: {self._account_number}\n"
            f"Holder Name: {self._holder_name}\n"
            f"Current Balance: ₹{self.__balance}"
        )

    @classmethod
    def change_minimum_balance(cls, amount: float) -> None:
        """
        Change minimum balance for all accounts.

        Parameters:
            amount (float): new minimum balance

        Returns:
            None
        """
        cls.minimum_balance = amount

    @staticmethod
    def bank_policy() -> str:
        """
        Returns bank policy.

        Returns:
            str: policy information
        """
        return "Minimum balance must be maintained in all savings accounts."

    @abstractmethod
    def calculate_interest(self) -> float:
        """
        Abstract method to calculate interest.

        Returns:
            float: interest amount
        """
        pass

class SavingsAccount(Account):
    """
    Savings account with interest feature.
    """

    interest_rate = 0.05

    def calculate_interest(self) -> float:
        """
        Calculate interest for savings account.

        Returns:
            float: interest amount
        """
        return self.get_balance() * self.interest_rate


class CurrentAccount(Account):
    """
    Current account with overdraft facility.
    """

    overdraft_limit = 20000

    def withdraw(self, amount: float) -> None:
        """
        Withdraw with overdraft facility.

        Parameters:
            amount (float): amount to withdraw

        Returns:
            None

        Raises:
            InsufficientBalanceError
        """
        if amount <= 0:
            raise ValueError("Withdraw amount must be positive")

        if self._get_balance() + self.overdraft_limit < amount:
            raise InsufficientBalanceError("Overdraft limit exceeded")

        self._set_balance(self._get_balance() - amount)
        self._transactions.append(
            f"{datetime.now()} - Withdrawn {amount} (Overdraft Allowed)"
        )

    def calculate_interest(self) -> float:
        """
        Current accounts do not provide interest.

        Returns:
            float: always 0
        """
        return 0.0


class Bank:
    """
    Bank class manages multiple accounts.
    """

    def __init__(self) -> None:
        """
        Initializes Bank with empty account storage.
        """
        self._accounts = {}

    def create_account(self, acc_type: str, acc_no: int,
                       name: str, balance: float) -> None:
        """
        Create new bank account.

        Parameters:
            acc_type (str): "savings" or "current"
            acc_no (int): account number
            name (str): holder name
            balance (float): initial balance

        Returns:
            None
        """
        if acc_type.lower() == "savings":
            account = SavingsAccount(acc_no, name, balance)
        elif acc_type.lower() == "current":
            account = CurrentAccount(acc_no, name, balance)
        else:
            raise ValueError("Invalid account type")

        self._accounts[acc_no] = account
        logging.info(f"{acc_type} account created for {name}")

    def get_account(self, acc_no: int) -> Account:
        """
        Retrieve account by account number.

        Parameters:
            acc_no (int): account number

        Returns:
            Account: account object
        """
        return self._accounts.get(acc_no)

    def transfer(self, from_acc: int, to_acc: int, amount: float) -> None:
        """
        Transfer money between two accounts.

        Parameters:
            from_acc (int): sender account number
            to_acc (int): receiver account number
            amount (float): transfer amount

        Returns:
            None
        """
        sender = self.get_account(from_acc)
        receiver = self.get_account(to_acc)

        if not sender or not receiver:
            raise ValueError("Invalid account number")

        sender.withdraw(amount)
        receiver.deposit(amount)

        logging.info(
            f"₹{amount} transferred from {from_acc} to {to_acc}"
        )


if __name__ == "__main__":

    bank = Bank()

    bank.create_account("savings", 101, "Shuvadip", 40000)
    bank.create_account("current", 102, "Rahul", 20000)

    acc1 = bank.get_account(101)

    acc1.deposit(5000)
    acc1.withdraw(2000)

    print(acc1.account_details())
    print("\nTransactions:")
    for t in acc1.show_transactions():
        print(t)

    print("\nInterest:", acc1.calculate_interest())

Bank Name: Panjab National Bank
Account Number: 101
Holder Name: Shuvadip
Current Balance: ₹43000

Transactions:
2026-02-25 11:49:21.321071 - Deposited ₹5000
2026-02-25 11:49:21.321071 - Withdrawn ₹2000

Interest: 2150.0


In [2]:
from abc import ABC, abstractmethod


class BankAccount(ABC):
    """
    Abstract class defining common banking rules
    """

    def __init__(self, account_number, customer_name, date_of_birth, balance):
        self.customer_name = customer_name
        self.__date_of_birth = date_of_birth
        self.__account_number = account_number
        self.__balance = balance

    def get_balance(self):
        return self.__balance

    def _set_balance(self, amount):
        self.__balance = amount

    def _get_account_number(self):
        return self.__account_number

    def _get_dob(self):
        return self.__date_of_birth

    @abstractmethod
    def deposit(self, amount):
        pass

    @abstractmethod
    def withdraw(self, amount):
        pass

    def display_account_details(self):
        print("Account Number:", self.__account_number)
        print("Customer Name:", self.customer_name)
        print("Date of Birth:", self.__date_of_birth)
        print("Balance:", self.__balance)


class SavingsAccount(BankAccount):

    def __init__(self, acc_no, name, dob, balance, minimum_balance=1000):
        super().__init__(acc_no, name, dob, balance)
        self.minimum_balance = minimum_balance

    def deposit(self, amount):
        if amount > 0:
            new_balance = self.get_balance() + amount
            self._set_balance(new_balance)
            print("Deposited:", amount)
        else:
            print("Invalid deposit amount")

    def withdraw(self, amount):
        if amount <= 0:
            print("Invalid withdrawal amount")
            return

        if self.get_balance() - amount < self.minimum_balance:
            print("Cannot withdraw. Minimum balance must be maintained.")
        else:
            new_balance = self.get_balance() - amount
            self._set_balance(new_balance)
            print("Withdrawn:", amount)


class CurrentAccount(BankAccount):

    def __init__(self, acc_no, name, dob, balance, overdraft_limit=20000):
        super().__init__(acc_no, name, dob, balance)
        self.overdraft_limit = overdraft_limit

    def deposit(self, amount):
        if amount > 0:
            new_balance = self.get_balance() + amount
            self._set_balance(new_balance)
            print("Deposited:", amount)
        else:
            print("Invalid deposit amount")

    def withdraw(self, amount):
        if amount <= 0:
            print("Invalid withdrawal amount")
            return

        if self.get_balance() + self.overdraft_limit < amount:
            print("Overdraft limit exceeded.")
        else:
            new_balance = self.get_balance() - amount
            self._set_balance(new_balance)
            print("Withdrawn:", amount, "(Overdraft Allowed)")


if __name__ == "__main__":

    print("---- Savings Account ----")
    s = SavingsAccount(101, "Shuvadip", "01-01-2000", 5000)
    s.deposit(2000)
    s.withdraw(5500)
    s.display_account_details()

    print("\n---- Current Account ----")
    c = CurrentAccount(102, "Rahul", "05-06-1998", 3000)
    c.withdraw(15000)
    c.display_account_details()

---- Savings Account ----
Deposited: 2000
Withdrawn: 5500
Account Number: 101
Customer Name: Shuvadip
Date of Birth: 01-01-2000
Balance: 1500

---- Current Account ----
Withdrawn: 15000 (Overdraft Allowed)
Account Number: 102
Customer Name: Rahul
Date of Birth: 05-06-1998
Balance: -12000
